In [62]:


# finds prime factorization of x
def gn(x):

    # keep a list of primes, along with a list of exponents
    primes = []
    exponents = []


    x += 1

    while x != 1:

        # find the next prime
        next_prime = find_next_prime(primes)

        # exp
        exponents.append(0)
        primes.append(next_prime)


        while mod(x, primes[-1]) == 0:
            x /= primes[-1]
            exponents[-1] += 1

    # return list of exponents
    return exponents





def find_next_prime(list_of_primes):
    
    # first 10 primes
    starting_primes = [2,3,5,7,11,13,17,19,23,29]


    l = len(list_of_primes)

    if l < 10:
        return starting_primes[l]
    

    else:
        # start from the last prime. Add 2. See if this is prime.

        x = list_of_primes[-1]
        
        while True:
            x += 2
            flag = 0
            
            # check if this new number is divisable by any of the previous primes
            for prime in list_of_primes:
            
                if mod(x,prime) == 0:
                    flag = 1

            
            if flag == 0:
                return x



def mod(x, y):
    return x-(x//y)*y
    


# Turns each godel number into a code
def decode_rule(x):
    
    rule = dict()
    rule['reference'] = 0
    rule['variable'] = 0    
    rule['action'] = 0

    rule['reference'], r = find_rh_lh(x)
    rule['action'], rule['variable'] = find_rh_lh(r)

    return rule




def find_rh_lh(x):

    lh = 0
    rh = 0
    
    if x == 0:
        return lh, rh
    
    x += 1

    while mod(x, 2) == 0:
        x /= 2
        lh +=1

    rh = (x-1)/2

    return int(lh), int(rh)



# takes list of instructions, decodes each one, and prints them out. 
# If given a Godel Number, decodes the Godel number into a set of instructions and then does the same
def decode(rules = None, x = None):
    
    if x and not rules:
        rules = gn(x)

    program = []

    for rule in rules:
        decoded_rule = decode_rule(rule)
        program.append(decoded_rule)


    return program



# prints the list of instructions for the program
def print_program(list_of_rules):

    for rule in list_of_rules:
        
        reference = util_reference_print_function(rule['reference'])
        variable = util_variable_print_function(rule['variable'])

        if rule['reference'] != 0:

            if rule['action'] == 0:
                print(f'[{reference}] {variable} <- {variable}')
        
            elif rule['action'] == 1:
                print(f'[{reference}] {variable} <- {variable} + 1')
            
            elif rule['action'] == 2:
                print(f'[{reference}] {variable} <- {variable} - 1')
            
            else:
                reference = util_reference_print_function(rule['action'] - 2)
                print(f'[{reference}] IF {variable} != 0 GOTO {reference}')
        
        else:

            if rule['action'] == 0:
                print(f'{variable} <- {variable}')
        
            elif rule['action'] == 1:
                print(f'{variable} <- {variable} + 1')
            
            elif rule['action'] == 2:
                print(f'{variable} <- {variable} - 1')
            
            else:
                reference = util_reference_print_function(rule['action'] - 2)
                print(f'IF {variable} != 0 GOTO', reference)
        

def find_variable_index(variable_index):
    """returns index of variable AND 0 if variable is input, 1 if variable is local, 2 if variable is output"""
    # map variable to index where X1...Xn are inputs, Z1,...,Zn are locals
    # if variable is input

    # y
    if variable_index == 0:
        return 0, 2

    # local
    if mod(variable_index, 2) == 0:
        return int(variable_index/2 - 1), 1
    
    # input
    else:
        return int((variable_index-1)/2), 0
    



def util_reference_print_function(reference):
    if reference == 0:
        return ''

    index = 1 + reference//5 
    letter = mod(reference,5)
    letters = ['A', 'B', 'C', 'D', 'E']

    return f'{letters[letter]}{index}'


def util_variable_print_function(variable):
    index, variable_type = find_variable_index(variable)
    
    index = int(index) + 1
    
    if variable_type == 0:
        return f'X{index}'
    
    if variable_type == 1:
        return f'Z{index}'
    
    if variable_type == 2:
        return f'Y'


In [60]:
program = decode([21, 46])
print_program(program)


[A1] X1 <- X1 + 1
IF X1 != 0 GOTO A1


In [14]:
program = decode([45,34,350,2,46])
print(program[4])
print_program(program)


{'reference': 1, 'variable': 1, 'action': 2}
[A1] X1 <- X1 - 1
[E1] Z2 <- Z2 + 1
IF X3 != 0 GOTO B1
[E1] Y <- Y + 1
[A1] X1 <- X1 - 1


In [63]:
def arg(input_list, element):

    for i in range(len(input_list)):
        if input_list[i] == element:
            return i

    return RuntimeError

    



def print_snapshot(snapshot):
    inputs = snapshot['inputs']
    locals = snapshot['locals']
    y = snapshot['y']

    # argument of snapshot reference in the references
    reference = snapshot['reference']


    output = [arg(snapshot['references'], reference) + 1] + inputs + locals + [y]

    output_string = ''

    for n in output:
        output_string += str(n)     
        output_string += ' '

    print(output_string)


# take input
line1 = input()
line2 = input()


instructions = line1.split(' ')
instructions = [int(instruction) for instruction in instructions]

inputs = line2.split(' ')
inputs = [int(input_value) for input_value in inputs]

program = decode(instructions)



# define snapshot
snapshot = dict()
snapshot['reference'] = 1
snapshot['inputs'] = []
snapshot['locals'] = []
snapshot['y'] = 0
snapshot['references'] = []


for input_value in inputs:
    snapshot['inputs'].append(input_value)



# if variable or reference is not there, add it to the list of variables
i = 1

max_local_index = 0
max_input_index = len(snapshot['inputs'])-1

for rule in program:

    variable_index, variable_type = find_variable_index(rule['variable'])

    if variable_type == 0 and variable_index>max_input_index:
        max_input_index = variable_index

    if variable_type == 1 and variable_index>max_local_index:
        max_local_index = variable_index
        
    snapshot['references'].append(rule['reference'])



for i in range(max_local_index + 1):
    snapshot['locals'].append(0)

for j in range(max_input_index):
    if j+1>=len(inputs):
        snapshot['inputs'].append(0)



print_program(program)
print("")


instruction_index = 0
x = 0

while True:

    # current snapshot
    if snapshot['reference'] not in snapshot['references']:
        print(snapshot['reference'])
        break


    print_snapshot(snapshot)

    rule = program[instruction_index]


    # type is 0, 1, 2
    variable_index, variable_type = find_variable_index(rule['variable'])
    
    variable_value = 0
    if variable_type == 0:
        variable_value = snapshot['inputs'][variable_index-1]

    elif variable_type == 1:
        variable_value = snapshot['locals'][variable_index-1]


    else:
        variable_value = snapshot['y']


    if rule['action'] in [0, 1, 2]:


        # update variable
        if rule['action'] == 0:
            pass

        elif rule['action'] == 1:
            if variable_type == 0:
                snapshot['inputs'][variable_index] += 1

            elif variable_type == 1:
                snapshot['locals'][variable_index] += 1

            else:
                snapshot['y'] += 1

        elif rule['action'] == 2:
            if variable_type == 0:
                snapshot['inputs'][variable_index] -= 1

            elif variable_type == 1:
                snapshot['locals'][variable_index] -= 1

            else:
                snapshot['y'] -= 1



        
        # Next Line
        if instruction_index + 1 < len(program):
            snapshot['reference'] = snapshot['references'][instruction_index+1]
            instruction_index += 1

        else:
            snapshot['reference'] = 'E'


    if rule['action'] > 2: 

        # GOTO
        if rule['action'] != 0:

            if variable_value != 0:
                snapshot['reference'] = rule['action'] - 2
                instruction_index = arg(snapshot['references'], snapshot['reference'])


        # Next Line
        else:
            if instruction_index + 1 < len(snapshot['references']):
                snapshot['reference'] = snapshot['references'][instruction_index+1]
                instruction_index += 1

            else:
                snapshot['reference'] = 'E'






[A1] X1 <- X1 - 1
Z2 <- Z2 + 1
IF X3 != 0 GOTO B1
Y <- Y + 1
IF X1 != 0 GOTO A1

1 2 1 0 0 0 0 
2 1 1 0 0 0 0 
2 1 1 0 0 1 0 
reference not in the list
2
